In [72]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report
import joblib
from sklearn.linear_model import LogisticRegression

In [73]:
# Doc du lieu
df = pd.read_csv("Crime_Data_from_2020_to_Present.csv", sep=",")

In [74]:
# Lay thuoc tinh va nhan
data = df[
    ['TIME OCC', 'AREA', 'Vict Age', 'Vict Sex', 'Vict Descent', 'Premis Cd', 'Weapon Used Cd', 'LOCATION', 'Crm Cd']]

In [75]:
# Ma hoa nhan
le = LabelEncoder()
data['Vict Sex'] = le.fit_transform(data['Vict Sex'])
data['Vict Descent'] = le.fit_transform(data['Vict Descent'])
data['AREA'] = le.fit_transform(data['AREA'])
data['Premis Cd'] = le.fit_transform(data['Premis Cd'])
data['LOCATION'] = le.fit_transform(data['LOCATION'])

C:\Users\ASUS\AppData\Local\Temp\ipykernel_25804\1477389903.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Vict Sex'] = le.fit_transform(data['Vict Sex'])
C:\Users\ASUS\AppData\Local\Temp\ipykernel_25804\1477389903.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Vict Descent'] = le.fit_transform(data['Vict Descent'])
C:\Users\ASUS\AppData\Local\Temp\ipykernel_25804\1477389903.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

In [76]:
# Scale du lieu
scaler = MinMaxScaler()
data[['Premis Cd', 'Vict Age', 'LOCATION']] = scaler.fit_transform(data[['Premis Cd','Vict Age', 'LOCATION']])

C:\Users\ASUS\AppData\Local\Temp\ipykernel_25804\2598005803.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[['Premis Cd', 'Vict Age', 'LOCATION']] = scaler.fit_transform(data[['Premis Cd','Vict Age', 'LOCATION']])


In [77]:
# Chuan hoa thoi gian
def categorize_time_period(time_occ):
    if 600 <= time_occ <= 1800:  # Daytime (6 AM to 6 PM)
        return 0
    else:  # Nighttime (6 PM to 5:59 AM)
        return 1


# Apply the function to the 'TIME OCC' column
data['TIME OCC'] = data['TIME OCC'].apply(categorize_time_period)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_25804\728664311.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['TIME OCC'] = data['TIME OCC'].apply(categorize_time_period)


In [78]:
data.head()

,TIME OCC,AREA,Vict Age,Vict Sex,Vict Descent,Premis Cd,Weapon Used Cd,LOCATION,Crm Cd
0,1,6,0.032258,3,12,0.000000,NaN,0.326116,510
1,0,0,0.411290,3,12,0.085987,NaN,0.028042,330
2,0,2,0.185484,4,18,0.417197,NaN,0.201078,480
3,1,8,0.185484,3,12,0.369427,NaN,0.204061,343
4,0,5,0.258065,3,7,0.003185,NaN,0.323693,354


In [79]:
data = data.dropna()

In [80]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 325476 entries, 9 to 966950
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   TIME OCC        325476 non-null  int64  
 1   AREA            325476 non-null  int64  
 2   Vict Age        325476 non-null  float64
 3   Vict Sex        325476 non-null  int32  
 4   Vict Descent    325476 non-null  int32  
 5   Premis Cd       325476 non-null  float64
 6   Weapon Used Cd  325476 non-null  float64
 7   LOCATION        325476 non-null  float64
 8   Crm Cd          325476 non-null  int64  
dtypes: float64(4), int32(2), int64(3)
memory usage: 22.3 MB


In [61]:
#data = data.drop('Weapon Used Cd', axis=1)

In [81]:
# Chuyen du lieu thanh dang array
X = data.iloc[:, : -1].values
y = data.iloc[:, -1].values

In [82]:
# Phan chia tap du lieu
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [83]:
# Tien hanh xu ly nhung thuoc tinh chua gia tri null
imputer = SimpleImputer(missing_values=np.nan, strategy="most_frequent")
X[:, 6:7] = imputer.fit_transform((X[:, 6:7]))

In [84]:
# Tao model
model = LogisticRegression(solver='newton-cholesky', max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='newton-cholesky')

In [85]:
# Make prediction on the test set
y_predict = model.predict(X_test)

In [86]:
acc = accuracy_score(y_test, y_predict)
print("Accuracy: ", acc)

print("Classification report on test set:")
print(classification_report(y_test, y_predict))


Accuracy:  0.38421715620007374
Classification report on test set:
              precision    recall  f1-score   support

         110       0.00      0.00      0.00       327
         121       0.00      0.00      0.00       714
         122       0.00      0.00      0.00        54
         210       0.31      0.14      0.20      6477
         220       0.00      0.00      0.00       945
         230       0.46      0.59      0.52     10611
         231       0.00      0.00      0.00       213
         235       0.00      0.00      0.00       123
         236       0.00      0.00      0.00      2517
         237       0.00      0.00      0.00        38
         250       0.00      0.00      0.00        89
         251       0.23      0.08      0.12       336
         310       0.31      0.19      0.24      1507
         320       0.00      0.00      0.00       115
         330       0.00      0.00      0.00       671
         331       0.00      0.00      0.00        83
         341   

c:\Users\ASUS\anaconda3\envs\TriTueNhanTao\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ASUS\anaconda3\envs\TriTueNhanTao\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ASUS\anaconda3\envs\TriTueNhanTao\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

In [87]:
# Luu model
joblib.dump(model, 'LR.pkl')

['LR.pkl']